# Combining large ocean-parcels datasets and choosing an optimal chunking. 

You might imagine that if you followed the instructions [on the making of parallel runs](https://github.com/OceanParcels/parcels/blob/dump_to_zarr/parcels/examples/documentation_MPI.ipynb) and the loading of the resulting dataset, you could just use the `dataset.to_zarr()` function to save the data to a single zarr datastore. This is true for small enough datasets -- but for a number of reasons, including the non-contiguous trajectory coordinate, if you try this with datasets whose size is larger than a 1/3 or so of the memory of your computer, you will find that it takes a long time, then fails as it exhausts the memory of your computer. 

This may be fixed in xarray or zarr in the future, but for now, we can work around this problem by saving the output in steps. 

At the same time, we can change the datatype of the output and modify the chunking of the dataset. Both of these can improve both performance and the size of the data on the disk. However, some care is required to do this, and this will be described in more detail below. 

## Why are we doing this? And what chunk sizes should we choose?

If you are running a relatively small case (perhaps 1/10 the size of the memory of your machine), nearly anything you do will work. However, as your problems get larger, it can help to write the data into a single zarr datastore, and to chunk that store appropriately. 

To illustrate this, here is the time it takes to retrieve all the results (with `ds['variableName'].values`) of some common data structures with different chunk sizes. (What is a chunk size? More on that below). The data in this example has 39 million trajectories starting over 120 times, and there are 250 observations, resulting in a directory size of 88Gb in double precision and 39 in single. In this table, "trajectory:5e4, obs:10" indicates that each chunk extends over 50,000 trajectories and 10 obs. The chunking in the original data is roughly a few thousand observations and 10 obs. 

|File type|open [s]|read 1 obs, all traj [s]|read 23 obs, all traj [s]|read 8000 contigous traj, all obs [s]|read traj that start at a given time, all obs [s]|
|---|---|---|---|---|---|
|Straigth from parcels|2.9|8.4|59.9|1.5|17.4|
|trajectory:5e4, obs:10|0.48|2.5|19.5|0.4|10.33|
|trajectory:5e4, obs:100|0.55|20.5|13.8|0.5|3.88|
|trajectory:5e5, obs:10|0.54|2.2|16.3|0.85|18.5|
|trajectory:5e5, obs:100|0.46|19.9|40.0|0.62|49.36|


You can see several things in this. It is always quicker to open a single file, and for all data access patterns, there is are chunksizes that are more efficient than the default output. Why is this?

The chunksize determines how data is stored on disk. For the default Zarr datastore, each chunk of data is stored as a single compressed file. In netCDF, chunking is similar except that the compressed data is stored within a single file. In either case, if you must access any data from within a chunk, you must read the entire chunk from disk. 

So when we access one obs dimension and many trajectories, the chunking scheme that is elongated in the trajectory direction is fastest. When we get all the observation for a scattered set of trajectories, the chunking that is elongated in observations is the best. In general, the product of the two chunksizes (the number of data points in a chunk) should be hundreds of thousands to 10s of millions.  A suboptimal chunking scheme is usually not tragic, but if you know how you will most often access the data, you can save considerable time. 

## How to save the output of an MPI ocean parcels run to a single zarr dataset

First, we need to import the necessary modules, specify the directory `inputDir` which contains the output of the parcels run (the directory that has proc01, proc02 and so forth), the location of the ouput zarr file `outputDir` and a dictionary giving the chunk size for the `trajectory` and `obs` coordinates, `chunksize`. 

In [1]:
import xarray as xr
from pylab import *
from numpy import *
from glob import glob
from os import path
import time

#first specify the directory in which the MPI code wrote its output
inputDir=('dataPathsTemp/'+
         'theAmericas_wholeGlobe_range100km_depthFrom_1m_to_500m_habitatTree_months01_to_02_fixed_1m/'+
         '2008/tracks.zarr')


#specify chunksize and where the output zarr file should go; also set chunksize of output file
chunksize={'trajectory':5*int(1e4),'obs':10}; 
outputDir='/home/pringle/jnkData/singleFile_5e4_X_10_example.zarr'

Now for large datasets, this code can take a while to run; for 36 million trajectories and 250 observations, it can take an hour and a half. I prefer not to accidently destroy data that takes more than an hour to create, so I put in a safety check and only let the code run if the output directory does not exist. 

In [2]:
#do not overwrite existing data sets
if path.exists(outputDir):
    print('the ouput path',outputDir,'exists')
    print('please delete if you want to replace it')
    assert False,'stopping execution'

It will often be useful to change the [`dtype`](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html) of the output data. Doing so can save a great deal of disk space. For example, the input data for this example is 88Gb in size, but by changing lat, lon and z to single precision, I can make the file about half as big. 

This comes at the cost of some accuracy. Float64 has 14 digits of accuracy, float32 has 7. For latitude and longitude, going from float64 to float32 increases the error by the circumfrence of the Earth devided 1e7, or about 1m. This is good enough for what I am doing. However, a year of time has about 3.15e7 seconds, and we often want to know within a second when a particle is released (to avoid floating point issues when picking out particles that start at a specific time). So the 3.15e7/1e7 error (a few seconds) in the time coordinate could cause problems. So I don't want to reduce the precision of time. 

To change precision, put an entry into the dictionary `varType` whose key is the name of the variable, and whose value is the type you wish the variable to be cast to:

In [3]:
varType={
         'lat':dtype('float32'),
         'lon':dtype('float32'),
          'time':dtype('datetime64'),
         'z':dtype('float32'),
         }

Now we need to read in the data as discussed in the section on making an MPI run:

In [7]:
print('opening data from multiple process files')
tic=time.time()
files = glob(path.join(inputDir, "proc*")); 
dataIn = xr.concat([xr.open_zarr(f) for f in files], dim='trajectory', 
                     compat='no_conflicts', coords='minimal') 
print('   done opening in %5.2f'%(time.time()-tic))

opening data from multiple process files
   done opening in  2.19


Now we can take advantage of the `.astype` operator to change the type of the variables. This is a lazy operator, and it will only be applied to the data when the data values are requested below, when the data is written to a new zarr store. 

In [8]:
for v in varType.keys():
    dataIn[v]=dataIn[v].astype(varType[v])

The dataset is then rechunked to our desired shape. This does not actually do anything right now, but will when the data is written below. Before doing this, it is useful to remove the per-variable chunking metadata, because of inconsistencies which arrise due to (I think) each MPI process output having a different chunking. This is explained in more detail in https://github.com/dcs4cop/xcube/issues/347  

In [9]:
print('re-chunking')
tic=time.time()
for v in dataIn.variables:
    if 'chunks' in dataIn[v].encoding:
        del dataIn[v].encoding['chunks']
dataIn=dataIn.chunk(chunksize)
print('   done in',time.time()-tic)

re-chunking
   done in 3.4046807289123535


The dataset `dataIn` is now ready to be written back out in stages. But how big is each segment that will be written out? I have found that writing out about 650 million points per variable is easy on a laptop with 16Gb of memory.  Larger sizes might be more efficient, but one does run into the law of diminishing returns fairly quickly. The number of points to write out is called `nPointsWrite` and it is integer devided by the `obs` dimension to get the number of rows to write out, `writeChunkLen`:

In [10]:
nPointsWrite=int(5e6)*130
writeChunkLen=nPointsWrite//dataIn.dims['obs']

Now, at some point in the future, one could imagine making this code parallel; to make that easier, make sure that writeChunkLen is an even multiple of the trajectory chunk size, and also make sure it is at least one.

In [11]:
writeChunkLen=(writeChunkLen//chunksize['trajectory'])*chunksize['trajectory']
writeChunkLen=max(1,writeChunkLen) #must be at least 1!
print("   writing %d trajectory's at a time"%(writeChunkLen,))

   writing 2600000 trajectory's at a time


Now we want to make a list of the indices that are to be written out, `chunkList`, so it goes from 0 to the total size of the trajectory dimension:

In [12]:
#make list of indices to write out, inclusive of end of the trajectory dimension
chunkList=list(range(0,dataIn.dims['trajectory'],writeChunkLen))
if chunkList[-1]!=dataIn.dims['trajectory']:
    chunkList.append(dataIn.dims['trajectory'])

Now iterate through the indices in chunkList, and write out each bit. The following code is a bit chatty, but as the code can take a while to run, I find it reassuring to know it is doing something. If one errs in using xarray, it can take an excessive amount of time to do things, so it is worth keeping an eye on things. After you start the code below running, go teach a class or revise a manuscript. It can take a while. On my test 88Gb file, it takes about an hour.

In [13]:
#write out the chunks
print('starting to write to',outputDir)
for nChunk in range(len(chunkList)-1):
    innerTic=time.time()
    print('   getting chunk',nChunk)
    newds=dataIn.isel(trajectory=np.arange(chunkList[nChunk],chunkList[nChunk+1])) #do not sort by trajectory
    print('   starting to write')
    if nChunk==0:
        newds.to_zarr(outputDir,mode='w')
    else:
        newds.to_zarr(outputDir,append_dim='trajectory')
    print('   done with chunk %d of %d from'%(nChunk,len(chunkList)-2),
          chunkList[nChunk],'to',chunkList[nChunk+1],'in',time.time()-innerTic)

print('Done writing in %5.1f'%(time.time()-tic,))

starting to write to /home/pringle/jnkData/singleFile_5e4_X_10_example.zarr
   getting chunk 0
   starting to write
   done with chunk 0 of 15 from 0 to 2600000 in 484.8957693576813
   getting chunk 1
   starting to write
   done with chunk 1 of 15 from 2600000 to 5200000 in 490.55356001853943
   getting chunk 2
   starting to write
   done with chunk 2 of 15 from 5200000 to 7800000 in 550.8107087612152
   getting chunk 3
   starting to write
   done with chunk 3 of 15 from 7800000 to 10400000 in 484.8843400478363
   getting chunk 4
   starting to write
   done with chunk 4 of 15 from 10400000 to 13000000 in 440.20081901550293
   getting chunk 5
   starting to write
   done with chunk 5 of 15 from 13000000 to 15600000 in 344.02604961395264
   getting chunk 6
   starting to write
   done with chunk 6 of 15 from 15600000 to 18200000 in 337.93852186203003
   getting chunk 7
   starting to write
   done with chunk 7 of 15 from 18200000 to 20800000 in 455.27793192863464
   getting chunk 8
 

We can now load the zarr data set we have created, and see what is in it, compared to what was in the input dataset. 

In [14]:
dataOriginal=dataIn = xr.concat([xr.open_zarr(f) for f in files], dim='trajectory', 
                     compat='no_conflicts', coords='minimal') 
dataProcessed=xr.open_zarr(outputDir)
print('The original data\n',dataOriginal,'\n\nThe new dataSet\n',dataProcessed)

The original data
 <xarray.Dataset>
Dimensions:     (trajectory: 39692941, obs: 250)
Coordinates:
  * obs         (obs) int32 0 1 2 3 4 5 6 7 ... 242 243 244 245 246 247 248 249
  * trajectory  (trajectory) int64 16 23 68 165 ... 39692792 39692889 39692920
Data variables:
    age         (trajectory, obs) float32 dask.array<chunksize=(3938, 10), meta=np.ndarray>
    lat         (trajectory, obs) float64 dask.array<chunksize=(3938, 10), meta=np.ndarray>
    lon         (trajectory, obs) float64 dask.array<chunksize=(3938, 10), meta=np.ndarray>
    time        (trajectory, obs) datetime64[ns] dask.array<chunksize=(3938, 10), meta=np.ndarray>
    z           (trajectory, obs) float64 dask.array<chunksize=(3938, 10), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.6/CF-1.7
    feature_type:           trajectory
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_mesh:           spherical
    parcels_version:        2.3.2.dev137 

The new dataSet
 <xar